In [3]:
import oracledb
import pandas as pd
import os
import sys
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Set working directory
os.chdir(r"C:\Wk 2 Mobile Banking App Review Analysis\Customer-Experience-Analytics")
sys.path.append(os.getcwd())

# Database connection details
hostname = 'localhost'
port = 1521
sid = 'bankreviews'
username = 'SYSTEM'
password = 'Opass#25'
dsn = oracledb.makedsn(hostname, port, sid=sid)

# Establish connection to Oracle Database
connection = oracledb.connect(user=username, password=password, dsn=dsn)

# Create a cursor object
cursor = connection.cursor()

# File paths for each bank's processed reviews
banks = {
    "CBE": "notebooks/data/processed_reviews_CBE.csv",
    "BOA": "notebooks/data/processed_reviews_BOA.csv",
    "Dashen": "notebooks/data/processed_reviews_Dashen.csv"
}

# Initialize an empty DataFrame to combine reviews
combined_reviews = pd.DataFrame()

# Load and combine reviews from CSV files
for bank, path in banks.items():
    bank_df = pd.read_csv(path)
    combined_reviews = pd.concat([combined_reviews, bank_df], ignore_index=True)

# Convert 'date' column to datetime format
combined_reviews['date'] = pd.to_datetime(combined_reviews['date'], errors='coerce')

# Ensure 'processed_review' column is filled and in the correct type
if 'processed_review' not in combined_reviews.columns or combined_reviews['processed_review'].isnull().any():
    combined_reviews['processed_review'] = combined_reviews['processed_review'].fillna('')

combined_reviews['processed_review'] = combined_reviews['processed_review'].astype(str)

# Insert combined reviews into the database
for index, row in combined_reviews.iterrows():
    cursor.execute("""
        INSERT INTO reviews (review_text, rating, review_date, bank_name, source, processed_review, sentiment_score, sentiment)
        VALUES (:1, :2, :3, :4, :5, :6, :7, :8)
    """, (row['review_text'], row['rating'], row['date'], row['bank_name'], row['source'], row['processed_review'], row['sentiment_score'], row['sentiment']))

# Commit the transaction
print("data stored to  oracledatabase successfully")
connection.commit()

# Close cursor and connection
cursor.close()
connection.close()





data stored to  oracledatabase successfully
